In [ ]:

"""
A producer P over a buffer list B is a set of workers P(W), a single job prod:(source, target) and holds a function 
ready signal s_p : B -> {0,1} indicating the buffers that need to be produced at the current time. 

it has states P.free, P.waiting, P.producing. 

A consumer C over that same buffer list B is also a set of workers and single job, and holds a function filled signal s_c : B->{0,1}
indicating the buffers that are filled and ready to consumed 

it has states C.free, C.waiting, C.consuming. 

a buffer b itself, is again a set/sequence of memory objects, (maybe some contigous chunk of memory)

and it has states b.empty, b.getting_produced, b.full, b.getting_consumed.  


okay, now let P be a producer, and C a consumer and B a list of buffers 

lets write some basic rules 

1. C.state(t+1) == C.waiting if  for all b in B, b.state(t) != b.full. 

2. P.state(t+1) == P.waiting if  for all b in B, b.state(t) != b.empty. 

3. The other is of course mutual exclisivity of states, a thing cannot be in two states at once 

4. P.state(t+1) = P.producing if for some b in B, b.state(t) = b.empty, in which case that b goes to b.state(t+1) = b.getting_produced
    and p.state(t) != p.producing. 
    
5  p.state(t+1) = P.free if P.state(t) = P.producing, in which case, the one b for which b.state(t) = b.getting_produced 
  now bas b.state(t+1) = b.full. 
  
6 C.state(t+1) = C.consuming if for some b in B, b.state(t) = b.full, in which case that b goes to b.state(t+1) = b.getting_consumed
   and C.state(t) != c.consuming 
7 C.state(t+1) = C.free if C.state(t) = C.consuming in which case the one b for which b.state(t) = b.getting_consumed 
now has state b.state(t_1) = b.empty. 



The init state is (P.free, C.free, bi = bi.empty for all bi in B)




"""

#### Correctness spec: 

we have a producer $P$ a consumer $C$ and a buffer set $B$. 
We have states 

$S(P) = \{p.f,s.w,p.pr\}$, $S(C) = \{c.f, c.w,c.cs\}$, $s(b) = \{e, gp, fl, gc\}$

###### System invariants: 

1. Mutual exclusivity, (implied by the set theory) any object can be in exactly one state at a given time. 
2. Producer lock: at any $t$, $S(P)_t = p.pr \iff !\exists b \in B: s(b)_t = gp$ 
3. Consumer lock: at any $t$  $S(C)_t = c.cs \iff !\exists b \in B: s(b)_t = gc$


##### Inital state: 

$(S(P)_0 = p.f, \ S(C)_0 = p.f, \  s(b) = e \  \forall b \in B$ 

##### tansitions 




In [ ]:
import numpy as np
class producer_consumer: 
  def __init__ (self, n_resources:int, n_producers:int, n_buffers:int, prod_job_latency:int, cons_job_latency:int, n_jobs): 
    """Here, we have n paralell resources of the same "size" in some sense (same number of threads say)
    we partition those into two parts, one part of producers, the other consumers. 
    we have a number of buffers, from which we can produce into and consume from, and we have some latency per resource 
    of both production and consumption, and it is obvoious that one resrouce handles the production/consumption of one job
    the goal is to compute consume(produce(job)) for n_jobs. 

    Args:
        n_resources (int): _description_
        n_producers (int): _description_
        n_buffers (int): _description_
        prod_job_latency (int): _description_
        cons_job_latency (int): _description_
        n_jobs (_type_): _description_
    """
    
    #we just creating as many states as there are atmost clock cycles which would just be (pl + cl + 1)*n_jobs
    self.pl = prod_job_latency 
    self.cl - cons_job_latency
    self.max_clocks = (self.pl + self.cl + 2)*n_jobs
    self.n_jobs = n_jobs 
    self.n_w = n_resources
    self.n_prod = n_producers 
    self.n_cons = self.n_w - self.n_prod
    self.n_buff = n_buffers
    
    self.producers_states = np.zeros((self.n_prod, self.max_clocks)).astype(int)
    self.consumers_states = np.zeros((self.n_cons,self.max_clocks)).astype(int)
    self.buffers_states = np.zeros((self.n_buff,n_jobs)).astype(int)
    
    self.producers_latency_points = np.zeros((self.n_prod)).astype(int)
    self.consumers_latency_points = np.zeros((self.n_cons)).astype(int)
    
    
    
    for i in range(1, self.max_clocks): 
      prev_empty_buffers = self.get_empty_buffers(self.buffers_states[i-1])
      prev_full_buffers = self.get_filled_buffers(self.buffers_states[i-1])
      prev_getting_produced_buffers = self.get_getting_produced_buffers(self.buffers_states[i-1])
      prev_getting_consumed_buffers = self.get_getting_consumed_buffers(self.buffers_states[i-1])
      prev_free_producers = self.get_free_producers(self.producers_states[i-1])
      prev_free_consumers = self.get_free_consumers(self.consumers_states[i-1])
      prev_waiting_producers = self.get_waiting_producers(self.producers_states[i-1])
      prev_waiting_consumers = self.get_waiting_consumers(self.consumers_states[i-1])
      prev_producing_producers = self.get_producing_producers(self.producers_states[i-1])
      prev_consuming_consumers = self.get_consuming_consumers(self.consumers_states[i-1])
      
  # --- Buffer State Getters ---
  # 0: empty
  # 1: getting_produced
  # 2: full
  # 3: getting_consumed

  def get_empty_buffers(self, buffers_state): 
    """Finds all buffers in the 'empty' (0) state."""
    return np.where(buffers_state == 0)

  def get_filled_buffers(self, buffers_state): 
    """Finds all buffers in the 'full' (1) state."""
    return np.where(buffers_state == 2)

  def get_getting_produced_buffers(self, buffers_state):
    """Finds all buffers in the 'getting_produced' (2) state."""
    return np.where(buffers_state == 1)

  def get_getting_consumed_buffers(self, buffers_state):
    """Finds all buffers in the 'getting_consumed' (3) state."""
    return np.where(buffers_state == 3)

  # --- Producer State Getters ---
  # 0: free
  # 1: waiting
  # 2: producing

  def get_free_producers(self, producers_state):
    """Finds all producers in the 'free' (0) state."""
    return np.where(producers_state == 0)

  def get_waiting_producers(self, producers_state):
    """Finds all producers in the 'waiting' (1) state."""
    return np.where(producers_state == 1)
    
  def get_producing_producers(self, producers_state): 
    """Finds all producers in the 'producing' (2) state."""
    return np.where(producers_state == 2)

  # --- Consumer State Getters ---
  # 0: free
  # 1: waiting
  # 2: consuming

  def get_free_consumers(self, consumers_state):
    """Finds all consumers in the 'free' (0) state."""
    return np.where(consumers_state == 0)

  def get_waiting_consumers(self, consumers_state):
    """Finds all consumers in the 'waiting' (1) state."""
    return np.where(consumers_state == 1)
    
  def get_consuming_consumers(self, consumers_state): 
    """Finds all consumers in the 'consuming' (2) state."""
    return np.where(consumers_state == 2)

In [ ]:
""" 
okay I guess we dont really need to maintain everything, what we will do 
is maintain a ready queue and wait queue as well as a "processing" counter for each producer and consumer 
and just maintain the number of empty, full, filling and consumed on buffers 


"""




In [ ]:
from collections import deque # <-- 1. Import deque

class Producer_Consumer:
  def __init__ (self, n_resources:int, n_producers:int, n_buffers:int, prod_job_latency:int, cons_job_latency:int, n_jobs):
    """Here, we have n paralell resources of the same "size" in some sense (same number of threads say)
    we partition those into two parts, one part of producers, the other consumers.
    we have a number of buffers, from which we can produce into and consume from, and we have some latency per resource
    of both production and consumption, and it is obvoious that one resrouce handles the production/consumption of one job
    the goal is to compute consume(produce(job)) for n_jobs.

    Args:
        n_resources (int): _description_
        n_producers (int): _description_
        n_buffers (int): _description_
        prod_job_latency (int): _description_
        cons_job_latency (int): _description_
        n_jobs (_type_): _description_
    """

    #we just creating as many states as there are atmost clock cycles which would just be (pl + cl + 1)*n_jobs
    self.pl = prod_job_latency
    self.cl = cons_job_latency # <-- 2. Fixed bug (was 'self.cl -')
    self.max_clocks = (self.pl + self.cl + 2)*n_jobs
    self.n_jobs = n_jobs
    self.n_w = n_resources
    self.n_prod = n_producers
    self.n_cons = self.n_w - self.n_prod
    self.n_buff = n_buffers

    # --- 3. Converted lists to deques ---
    # These are pools or queues, so deque is ideal.
    self.free_producers = deque([i for i in range(self.n_prod)])
    self.free_consumers = deque([i for i in range(self.n_cons)])
    self.waiting_consumers = deque()
    self.waiting_producers = deque()
    
    # These will likely store (id, finish_clock) tuples
    self.producing_producers = deque() 
    self.consuming_consumers = deque()
    # --- End of deque conversion ---

    self.n_empty_buffers = self.n_buff
    self.n_full_buffers = 0
    self.n_getting_filled_buffers = 0
    self.n_getting_consumed_buffers = 0

    # --- 4. Kept these as lists ---
    # These look like they are used as arrays (accessed by index),
    # so a standard list is the correct choice here.
    self.produces_latency_points = [0 for _ in range (self.n_prod)]
    self.consumer_latency_points = [0 for _ in range (self.n_cons)]

    timer = 0
    for i in range(self.max_clocks):
    #phase one, arrivals: 
      while (self.produces_latency_points[self.producing_producers[-1]] == self.pl -1) and self.producing_producers: 
        x = self.producing_producers.pop()
        self.produces_latency_points[x] = 0 
        self.n_full_buffers +=1 
        self.n_getting_filled_buffers -= 1
        self.free_producers.appendleft(x) #arrives 
        
      while (self.consumer_latency_points[self.consuming_consumers[-1]] == self.cl -1) and self.consuming_consumers: 
        x = self.consuming_consumers.pop()
        self.consumer_latency_points[x] = 0 
        self.n_empty_buffers +=1 
        self.n_getting_consumed_buffers -= 1
        self.free_consumers.appendleft(x) #arrives 
        self.n_jobs -= 1
        
      #phase 1.1 increments 
      for x in self.producing_producers: 
        self.produces_latency_points[x] += 1
      
      for x in self.consuming_consumers: 
        self.consumer_latency_points[x] += 1
        
      #phase 2 schedule new jobs 
      while(self.waiting_consumers) and (self.n_full_buffers > 0): 
        x = self.waiting_consumers.pop() 
        self.n_full_buffers -= 1 
        self.n_getting_consumed_buffers += 1
        self.consuming_consumers.appendleft(x) 
        
      while(self.waiting_producers) and (self.n_empty_buffers > 0): 
        x = self.waiting_producers.pop()
        self.n_empty_buffers -= 1
        self.n_getting_filled_buffers += 1 
        self.producing_producers.appendleft(x) 
        
      #phase3 free ones wait for new jobs 
      
      while(self.free_consumers): 
        x  = self.free_consumers.pop()
        self.waiting_consumers.appendleft(x)
        
      while (self.free_producers): 
        x = self.free_producers.pop()
        self.waiting_producers.appendleft(x)
        
        


In [ ]:
from collections import deque
import textwrap

class Producer_Consumer:
    def __init__(self, n_resources: int, n_producers: int, n_buffers: int, prod_job_latency: int, cons_job_latency: int, n_jobs: int):
        
        # --- Parameters ---
        self.pl = prod_job_latency
        self.cl = cons_job_latency
        self.n_jobs_total = n_jobs
        self.n_w = n_resources
        self.n_prod = n_producers
        self.n_cons = self.n_w - self.n_prod
        self.n_buff = n_buffers
        
        # Max clock safeguard to prevent infinite loops
        self.max_clocks = (self.pl + self.cl + 2) * self.n_jobs_total + 1000 # Added a buffer

        # --- Job Counters ---
        self.n_jobs_started = 0
        self.n_jobs_completed = 0

        # --- Worker Pools (using append/pop as stacks) ---
        self.free_producers = deque(range(self.n_prod))
        self.free_consumers = deque(range(self.n_cons))
        self.waiting_consumers = deque()
        self.waiting_producers = deque()
        
        # --- Active Worker Lists (using deque as a set/list) ---
        self.producing_producers = deque() 
        self.consuming_consumers = deque()
        
        # --- Buffer State Counters ---
        self.n_empty_buffers = self.n_buff
        self.n_full_buffers = 0
        self.n_getting_filled_buffers = 0
        self.n_getting_consumed_buffers = 0

        # --- Latency Tracking ---
        self.produces_latency_points = [0] * self.n_prod
        self.consumer_latency_points = [0] * self.n_cons
        
        # This will hold the history of our simulation
        self.state_history = []

    def _get_state_repr(self, timer: int) -> str:
        """Helper method to create a snapshot of the current state."""
        # Using textwrap to make the output clean
        state = f"""
        =====================================================
        TIMER: {timer}
        =====================================================
        JOBS:    {self.n_jobs_completed} / {self.n_jobs_total} Completed. ({self.n_jobs_started} Started)
        BUFFERS: Empty({self.n_empty_buffers}), Full({self.n_full_buffers}), Filling({self.n_getting_filled_buffers}), Consuming({self.n_getting_consumed_buffers})
        ---
        PRODUCERS:
          Free:     {list(self.free_producers)}
          Waiting:  {list(self.waiting_producers)}
          Producing: {list(self.producing_producers)}
        CONSUMERS:
          Free:     {list(self.free_consumers)}
          Waiting:  {list(self.waiting_consumers)}
          Consuming: {list(self.consuming_consumers)}
        """
        return textwrap.dedent(state)

    def run_simulation(self):
        """
        Runs the full simulation clock by clock.
        
        The order of operations in each clock cycle is critical:
        1. Check for finished work (Reap)
        2. Schedule new work (Sow)
        3. Assign new jobs to free workers
        4. Move any remaining free workers to 'waiting'
        5. Increment latency counters for active workers
        """
        timer = 0

        # Run until all jobs are *completed*, with a safeguard
        while self.n_jobs_completed < self.n_jobs_total:
            
            # Log the state *before* any changes in this cycle
            self.state_history.append(self._get_state_repr(timer))
            
            if timer > self.max_clocks:
                self.state_history.append("!!! SIMULATION TIMED OUT !!!")
                break
            
            # --- PHASE 1: CHECK FOR FINISHED WORK (REAP) ---
            # We iterate over a static list() copy to safely remove from the deque
            
            for producer_id in list(self.producing_producers):
                # Check if the job is done (latency counter is at max)
                if self.produces_latency_points[producer_id] == self.pl:
                    self.producing_producers.remove(producer_id)
                    self.produces_latency_points[producer_id] = 0 # Reset counter
                    
                    self.n_full_buffers += 1
                    self.n_getting_filled_buffers -= 1
                    self.free_producers.append(producer_id) # Add to FREE pool

            for consumer_id in list(self.consuming_consumers):
                if self.consumer_latency_points[consumer_id] == self.cl:
                    self.consuming_consumers.remove(consumer_id)
                    self.consumer_latency_points[consumer_id] = 0 # Reset counter
                    
                    self.n_empty_buffers += 1
                    self.n_getting_consumed_buffers -= 1
                    self.free_consumers.append(consumer_id) # Add to FREE pool
                    
                    self.n_jobs_completed += 1 # A job is fully done!

            # --- PHASE 2: SCHEDULE NEW WORK (SOW) ---
            # Match waiting workers to available buffers
            
            while self.waiting_producers and self.n_empty_buffers > 0:
                producer_id = self.waiting_producers.pop()
                
                self.n_empty_buffers -= 1
                self.n_getting_filled_buffers += 1
                self.producing_producers.append(producer_id) # Move to PRODUCING
                
            while self.waiting_consumers and self.n_full_buffers > 0:
                consumer_id = self.waiting_consumers.pop()
                
                self.n_full_buffers -= 1
                self.n_getting_consumed_buffers += 1
                self.consuming_consumers.append(consumer_id) # Move to CONSUMING

            # --- PHASE 3: ASSIGN NEW JOBS & MOVE FREE TO WAITING ---
            # This is your explicit model of the barrier `arrive` phase
            
            # First, assign new job "tickets" to free producers
            while self.free_producers and self.n_jobs_started < self.n_jobs_total:
                producer_id = self.free_producers.pop()
                self.waiting_producers.append(producer_id) # Move to WAITING
                self.n_jobs_started += 1
            
            # Second, move *all remaining* free workers to the waiting state
            # This models them "arriving" and now "waiting" for the next cycle
            while self.free_producers:
                self.waiting_producers.append(self.free_producers.pop())
                
            while self.free_consumers:
                self.waiting_consumers.append(self.free_consumers.pop())

            # --- PHASE 4: INCREMENT LATENCY COUNTERS ---
            # This must happen *last*, after all assignments are done.
            # We increment the timer for any worker that is *actively* working.
            
            for producer_id in self.producing_producers:
                self.produces_latency_points[producer_id] += 1
            
            for consumer_id in self.consuming_consumers:
                self.consumer_latency_points[consumer_id] += 1
                
            # --- END OF CYCLE ---
            timer += 1
        
        # Log the final state
        self.state_history.append(self._get_state_repr(timer))
        self.state_history.append(f"--- SIMULATION FINISHED AT CLOCK {timer} ---")
        
        return self.state_history

# --- Example of how to run it ---

# Create the simulation
sim = Producer_Consumer(
    n_resources=10, 
    n_producers=4,    # 4 producers
    n_buffers=6,      # 6 buffers
    prod_job_latency=5, # 5 clock cycles to produce
    cons_job_latency=3, # 3 clock cycles to consume
    n_jobs=20         # 20 total jobs
)

# Run it
history = sim.run_simulation()

# Print the first 10 and last 10 states
for state in history[:10]:
    print(state)

print("\n... [simulation running] ...\n")

for state in history[-10:]:
    print(state)


TIMER: 0
JOBS:    0 / 20 Completed. (0 Started)
BUFFERS: Empty(6), Full(0), Filling(0), Consuming(0)
---
PRODUCERS:
  Free:     [0, 1, 2, 3]
  Waiting:  []
  Producing: []
CONSUMERS:
  Free:     [0, 1, 2, 3, 4, 5]
  Waiting:  []
  Consuming: []


TIMER: 1
JOBS:    0 / 20 Completed. (4 Started)
BUFFERS: Empty(6), Full(0), Filling(0), Consuming(0)
---
PRODUCERS:
  Free:     []
  Waiting:  [3, 2, 1, 0]
  Producing: []
CONSUMERS:
  Free:     []
  Waiting:  [5, 4, 3, 2, 1, 0]
  Consuming: []


TIMER: 2
JOBS:    0 / 20 Completed. (4 Started)
BUFFERS: Empty(2), Full(0), Filling(4), Consuming(0)
---
PRODUCERS:
  Free:     []
  Waiting:  []
  Producing: [0, 1, 2, 3]
CONSUMERS:
  Free:     []
  Waiting:  [5, 4, 3, 2, 1, 0]
  Consuming: []


TIMER: 3
JOBS:    0 / 20 Completed. (4 Started)
BUFFERS: Empty(2), Full(0), Filling(4), Consuming(0)
---
PRODUCERS:
  Free:     []
  Waiting:  []
  Producing: [0, 1, 2, 3]
CONSUMERS:
  Free:     []
  Waiting:  [5, 4, 3, 2, 1, 0]
  Consuming: []


TIMER: 4
JO